# College Football Simulation with new pairing system
### By Rodrigo Vargas and Daniel Yedidia
Project for PIC-16B Python with Applications II, UCLA Spring 2023 <br>
Github: https://github.com/dyeds/PIC-16B-Project

Structure: (Tentative)

1. Information & Credits: CollegeFootballData.com, https://www.reddit.com/r/CFB/comments/qq553i/what_would_college_football_look_like_under_a/ Post by u/dethwing_.
2. Explanation of the project and how to use the notebook
3. Data Acquisition and Preprocessing. Working with CFBD and BingMaps API's and storing locally on Database
4. Model creation using Tensorflow. Creating various models and motives on why to use Betting Lines.
5. Explanation and Implementation of Minimum Weight Matching Algorithm using Networkx for pairings and using distances.
6. SQL Database with Simulated Games and other table creating Functions.
7. Visualizations of Simulation using Plotly.
8. Biases and Future Improvements for the project. 

## Information & Credits

## Project Explanations and Instructions

In [ ]:
#List of Imports here:
import cfbd
from cfbd.rest import ApiException


## Data Acquisition and Preprocessing

As explained before, we need College Football Data and Locations Data for the simulation, which will be obtained using CFBD and BingMaps API's. They typically require registering and getting an unique key, this is done to keep track of who and how many API calls are needed. Some API's may ask you to pay after certain number of calls, but for this project is not needed. <br>

College football data is needed for all the simulation purposes, extracting all FBS teams and their season stats. First register on https://collegefootballdata.com/key to obtain a key and creating a configuration object with that data, we will be able to use the API.

In [ ]:
# import cfbd
# from cfbd.rest import ApiException

configuration = cfbd.Configuration()
# configuration.api_key['Authorization'] = 'YOUR_KEY_HERE'
configuration.api_key['Authorization'] = '3WCU5V2X05Rvh60ZxUG8FarJN4s2D1lcd2c2r6Kz/qL1Y3tVBJtWsuNATnzHRV2h'
configuration.api_key_prefix['Authorization'] = 'Bearer'

Now we have to extract data from 3 classes: `GamesApi` which gives us data for all the games; `BettingApi` which gives us data for all the betting lines; and `StatsApi` which gives us the team season data.<br>

In our predictive modelling, we are going to use a merge between game-data and team-data as our predictors, and betting lines data as our target. Down below is an example for data extraction, more detail was done on functions inside `DataFunctions.py` file. 

In [ ]:
api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))
year=2022
division="fbs"

# try:
#     api_response = api_instance.get_games(year=year,division=division)
#     print(len(api_response))
# except:
#     print(ApiException)

## Predictive Model using Tensorflow

## Pairing System using Minimum Weight Matching Algorithm

## Simulated Results

## Graphical Visualizations of Results

## Further Improvements